# (A) Statewide Hexmap: Reading Data and Outputting Hex-Binned Files

A select list of cleaned and processed data used to power the SFR (Single Family Residential) investor project.

***NOTE: The data here is PRELIMINARY and are subject to change as additional verification, fact-checking, etc. takes place. If you spot any errors, please contact [Tyler Dukes](mtdukes@newsobserver.com).***

- **investors_labeled** A comma-separated value file of investor LLCs, DBAs and other entities and their variations linked to parent companies, based on a variety of sources. Filename includes datetime the list was last generated.
- **wake_sfr_investors** Comma-separated value file of Wake County properties joined with and filtered for all owners identified as investors. NOTE: list includes iBuyers. *Filename includes date the list was last generated.*
- **mecklenburg_sfr_investors** Comma-separated value file of Mecklenburg County properties joined with and filtered for all owners identified as investors. Filename includes datetime the list was last generated. *NOTE: List includes iBuyers.*
- **onemap_investors** Comma-separated value file of Mecklenburg County properties joined with and filtered for all owners identified as investors. Filename includes datetime the list was last generated. *NOTE: list includes iBuyers and may include properties that are not single-family homes.*

In [1]:
import os
import json
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import overpass
import geojson
import json
import subprocess

# To run these steps you need a local version of Mapshaper (mapshaper.org)
MAPSHAPER = os.path.join(os.path.expanduser('~'), "node_modules/mapshaper/bin/mapshaper")
DATA_DIRECTORY = os.path.join(os.getcwd(), 'data')

### (1) Reading the datasets

In [2]:
investors_labeled = pd.read_csv(os.path.join(
    DATA_DIRECTORY, "investors_labeled202204120828.txt"
))
mecklenburg_sfr_investors = pd.read_csv(os.path.join(
    DATA_DIRECTORY, "mecklenburg_sfr_investors202203211519.txt"
))
wake_sfr_investors = pd.read_csv(os.path.join(
    DATA_DIRECTORY, "wake_sfr_investors202203211516.txt"
))
onemap_investors = pd.read_csv(os.path.join(
    DATA_DIRECTORY, "corporate_sfr_properties202205041359.csv"
))
nc_investor_transactions = pd.read_csv(os.path.join(
    DATA_DIRECTORY,"nc_sfr_investor_transactions202204251343_prelim.txt"
))

### (1b) Outputting File for Searchable Map

In [3]:
onemap_investors['site_address'] = onemap_investors['site_address'].fillna('')

searchable_residences = {
    "type": "FeatureCollection",
    "features": []
}

for i, row in onemap_investors.iterrows():
    sf_id = row.sf_id
    lat = row.lat
    lng = row.lng
    address = row.site_address
    investor = row.investor_label_lvl2
    owner = row.owner_clean
#     zipcode = str(int(row.site_zip)) if ~np.isnan(float(row.site_zip)) else ''
    coordinates = [lng, lat]
    
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": coordinates
        },
        "properties": {
            "address": address,
#             "zip": zipcode,
            "investorId": investor + "-" + str(i),
            "investor": investor,
            "owner": owner
        }
    }
    searchable_residences['features'].append(feature)


In [5]:
searchable_residences_filename = 'searchable_residences.json'
searchable_residences_filepath = os.path.join(DATA_DIRECTORY, searchable_residences_filename)
    
with open(searchable_residences_filepath, 'w') as f:
    json.dump(searchable_residences, f)

### (2) Outputting a GeoJSON file for use in Hexbin Processing

In [7]:
investor_owned_residences = {
    "type": "FeatureCollection",
    "features": []
}

for i, row in onemap_investors.iterrows():
    sf_id = row.sf_id
    lat = row.lat
    lng = row.lng
    address = "{0}, {1}, NC {2}".format(row.site_address, row.site_city, row.site_zip)
    investor = row.investor_label_lvl2
    coordinates = [lng, lat]
    
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": coordinates
        },
        "properties": {
            "id": sf_id,
            "address": address,
            "investor": investor
        }
    }
    investor_owned_residences['features'].append(feature)

In [8]:
investor_owned_sfr_filename = 'investor_owned_residences.json'
investor_owned_sfr_filepath = os.path.join(DATA_DIRECTORY, investor_owned_sfr_filename)
    
with open(investor_owned_sfr_filepath, 'w') as f:
    json.dump(investor_owned_residences, f)

**Top 20:** Checking which cities are high on the list of institutionally owned single family residences

In [9]:
nc_investor_transactions.propertycity.value_counts()[1:20]

Raleigh          1581
Concord          1295
Winston Salem    1169
Clayton           991
Huntersville      806
Greensboro        753
Indian Trail      672
Monroe            665
Mooresville       634
Durham            508
Gastonia          474
High Pt           391
Matthews          377
Fuquay Varina     364
Garner            363
Waxhaw            353
Kannapolis        309
Kernersville      279
Wake Forest       266
Name: propertycity, dtype: int64

### (3a) Output hexagons at a statewide zoom level (hexSideLength = 2 miles) 

In order to run this step you need to be running `node` and have installed the `npm` packages required by the `hexbin-processing.js` script. You also need to have a local binary of `mapshaper` installed.

In [ ]:
hexagon_filename = "hexagonsGeo.json"
hexagon_filepath = os.path.join(DATA_DIRECTORY, hexagon_filename)
hexagon_topojson = os.path.join(DATA_DIRECTORY, "hexagons.json")
# Length of the side of the hexagon in miles
hex_side_length = 2

subprocess.run([
    "node",
    "hexbin-processing.js",
    "--input={0}".format(investor_owned_sfr_filename),
    "--output={0}".format(hexagon_filename),
    "--side={0}".format(hex_side_length)
], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

subprocess.run([
    MAPSHAPER,
    hexagon_filepath,
    "-rename-layers",
    "names=hexagons",
    "-o",
    "format=topojson",
    hexagon_topojson
], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

### (3b) Output hexagons at a more granular neighborhood zoom level (hexSideLength = 0.2 miles) 

In order to run this step you need to be running `node` and have installed the `npm` packages required by the `hexbin-processing.js` script. You also need to have a local binary of `mapshaper` installed.

In [ ]:
zoomed_hexagon_filename = "zoomedHexagonsGeo.json"
zoomed_hexagon_filepath = os.path.join(DATA_DIRECTORY, zoomed_hexagon_filename)
zoomed_hexagon_topojson = os.path.join(DATA_DIRECTORY, "zoomedHexagons.json")
# Length of the side of the hexagon in miles
zoomed_hex_side_length = 0.2

subprocess.run([
    "node",
    "hexbin-processing.js",
    "--input={0}".format(investor_owned_sfr_filename),
    "--output={0}".format(zoomed_hexagon_filename),
    "--side={0}".format(zoomed_hex_side_length)
], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

subprocess.run([
    MAPSHAPER,
    zoomed_hexagon_filepath,
    "-rename-layers",
    "names=hexagons",
    "-o",
    "format=topojson",
    zoomed_hexagon_topojson
], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# (B) Neighborhood Map: Join Investor Data w/ Residential Building Outlines

Here I'm taking OpenStreetMaps data, and combining it with both North Carolina parcel data and institutionally owned single-family resdience data. The end result is a topojson that has joined individual building outlines with the institution that owns it. There's a better way to do this I'm sure.

In [12]:
from shapely.geometry import shape, GeometryCollection, Polygon, MultiPolygon

# Overpass API isntance
OVERPASS_API = overpass.API()

# North Mecklenburg County bounding box
# Overpass API accepts [lat1, lng1, lat2, lng2] ordering
bbox = [35.28360260045482, -80.90555191040039, 35.304041146172075, -80.85420370101929]
bbox_string = ', '.join([str(num) for num in bbox])

# Need to switch and reorder the lat/lng
# Mapshaper API accepts [lngMin, latMin, lngMax, latMax]
bbox_xy = [min(bbox[1], bbox[3]), min(bbox[0], bbox[2]), max(bbox[1], bbox[3]), max(bbox[0], bbox[2])]
bbox_xy_string = ', '.join([str(num) for num in bbox_xy])

### (1) Get underlying geometries from Overpass API

Output TopoJSON files since those are relatively compressed for mapping purposes. This is not really about joining any data. However, these steps are necessary for mapping purposes.

In [13]:
import topojson as tp

def output_topojson(geojson, layer_name, property_fields=[]):
    topojson = tp.Topology(geojson, object_name=layer_name)
    topojson_output = json.loads(topojson.to_json())
    
    if len(property_fields) > 0:
        topojson_output = topojson.output
        geometries = topojson_output['objects'][layer_name]['geometries']
        
        for i, geom in enumerate(geometries):
            properties = geojson['features'][i]['properties']
            properties_to_keep = { field: properties[field] for field in property_fields }
            topojson_output['objects'][layer_name]['geometries'][i]['properties'] = properties_to_keep
    
    output_filepath = os.path.join(DATA_DIRECTORY, '{0}.json'.format(layer_name))

    with open(output_filepath, 'w') as f:
        json.dump(topojson_output, f)

overpass_highways = OVERPASS_API.get("""
( way["highway"="motorway"]({0});
  way["highway"="trunk"]({0});
  way["highway"="primary"]({0});
  way["highway"="secondary"]({0});
  way["highway"="tertiary"]({0});
)
""".format(bbox_string), verbosity='geom')

overpass_streets = OVERPASS_API.get("""
(
  way["highway"="service"]({0});
  way["highway"="residential"]({0});
)
""".format(bbox_string), verbosity='geom')

overpass_turning_circles = OVERPASS_API.get("""
(
  node["highway"="turning_circle"]({0});
)
""".format(bbox_string), verbosity='geom')

In [14]:
output_topojson(overpass_turning_circles, 'turning_circles')
output_topojson(overpass_streets, 'streets', ['highway'])
output_topojson(overpass_highways, 'highways', ['highway'])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/topojson/core/extract.py:403: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  self.records_collection = len(geom)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/topojson/core/extract.py:409: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for idx, geo in enumerate(geom):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/topojson/core/extract.py:182: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  self._extract_point(geom)
<__array_

### (2) Get residential buildings from Overpass API
API returns a FeatureCollection, a GeoJSON type

In [15]:
overpass_residential = OVERPASS_API.get("""
((way["building"]({0}); relation["building"]({0});););      
""".format(bbox_string), verbosity='geom')

### (3) Convert LineStrings to Polygons (assumes closed LineStrings)


In [16]:
shapely_features = []
for feature in overpass_residential.features:
    if feature.geometry.type == "MultiPolygon":
        polygons = [Polygon(p) for p in feature["geometry"]["coordinates"][0]]
        shapely_features.append(shape(MultiPolygon(polygons)).buffer(0))
    else:
        shapely_features.append(shape(Polygon(feature["geometry"]["coordinates"])).buffer(0))
    
shapely_residential = GeometryCollection(shapely_features)

### (4) Get the neighborhood outline shape
Manually draw neighborhood outline on https://geojson.io/

Download a geoJSON file named `outline_neighborhood.geojson`

In [17]:
neighborhood_filepath = os.path.join(DATA_DIRECTORY, 'outline_neighborhood.geojson')
with open(neighborhood_filepath) as f:
    features = json.load(f)["features"]

# NOTE: buffer(0) is a trick for fixing scenarios where polygons have overlapping coordinates 
outline = GeometryCollection([shape(feature["geometry"]).buffer(0) for feature in features])

### (5) Find all residential buildings contained in the neighborhood outline

In [18]:
neighborhood_buildings = []
for bldg in shapely_residential.geoms:
    # add intersection to the list
    if outline.contains(bldg):
        neighborhood_buildings.append(bldg)

### (6) Get tax parcels that interesect with neighborhood outline

Time consuming step. I could probably improve this one.

In [ ]:
parcel_taxdata_filepath = os.path.join(DATA_DIRECTORY, 'parcel_taxdata', 'Parcel_TaxData.shp')
parcel_projected_filepath = os.path.join(DATA_DIRECTORY, 'parcel_projected.json')

subprocess.run([
    MAPSHAPER,
    parcel_taxdata_filepath,
    "-proj",
    "wgs84",
    "-o",
    "format=geojson",
    parcel_projected_filepath
], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)


In [ ]:
neighborhood_parcel_filepath = os.path.join(DATA_DIRECTORY, 'parcel_neighborhood.json')

subprocess.run([
    MAPSHAPER,
    parcel_projected_filepath,
    "-clip",
    "bbox={0}".format(bbox_xy_string),
    "-o",
    "format=geojson",
    neighborhood_parcel_filepath
], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)


In [21]:
with open(neighborhood_parcel_filepath) as f:
    features = json.load(f)["features"]

neighborhood_parcels = GeometryCollection([shape(feature['geometry']).buffer(0) for feature in features])
neighborhood_properties = [feature['properties'] for feature in features]

### (6) Get tax parcels that interesect with residential buildings

Checking if a tax parcel contains the residential building centroid.

In [22]:
neighborhood_building_properties = []

for bldg in neighborhood_buildings:
    centroid = bldg.centroid
    for i, parcel in enumerate(neighborhood_parcels.geoms):
        if parcel.contains(centroid):
            neighborhood_building_properties.append(neighborhood_properties[i])
            break

### (7) Join investor ownership data with residential building feature

Uses parcel ID as the unique identifier and only keep the investor property.

In [23]:
final_properties = []
for i, properties in enumerate(neighborhood_building_properties):
    # parcel id
    pid = properties['pid']
    sfr_match = onemap_investors[onemap_investors.parcel_identification1 == pid]
    investor = ""
    if len(sfr_match) > 1:
        investor = list(sfr_match['investor_label_lvl2'])[0]
    elif len(sfr_match) > 0:
        investor = sfr_match['investor_label_lvl2'].item()
    final_properties.append({"investor": investor})

### (8) Convert Shapely geometries to geojson

In [24]:
import shapely.wkt
homes = {
    "type": "FeatureCollection",
    "features": []
}

for i, bldg in enumerate(neighborhood_buildings):
    feature = geojson.Feature(geometry=shapely.wkt.loads(bldg.wkt), properties=final_properties[i])
    homes['features'].append(feature)

In [25]:
output_topojson(homes, 'homes', ['investor'])

# (C) Timeline Map: Formatting NC Statewide Transaction Data

### (1) Format NC investor transaction data

In [26]:
from datetime import datetime, timezone

nc_investor_transactions = nc_investor_transactions.fillna("") 
nc_investor_transactions['year'] = nc_investor_transactions.saledate.str.split('-').str[0].astype(int)
nc_investor_transactions['month'] = nc_investor_transactions.saledate.str.split('-').str[1].astype(int)
nc_investor_transactions['day'] = nc_investor_transactions.saledate.str.split('-').str[2].astype(int)
nc_investor_transactions['saledate_ordered'] = nc_investor_transactions.saledate.str.replace('-','').astype(int)
nc_investor_transactions = nc_investor_transactions.sort_values(['saledate_ordered']).reset_index(drop=True)
nc_investor_transactions['lng_rounded'] = round(nc_investor_transactions.lng,2)
nc_investor_transactions['lat_rounded'] = round(nc_investor_transactions.lat,2)
nc_investor_transactions['timestamp'] = [datetime(row.year, row.month, row.day).replace(tzinfo=timezone.utc).timestamp() for i, row in nc_investor_transactions.iterrows()]
nc_investor_transactions['timestamp'] = nc_investor_transactions['timestamp'].astype(int)

### (2) Output timeline geojson data

In [27]:
timeline_geodata = {
    "type": "FeatureCollection",
    "features": []
}

def to_str_int(string):
    if not string:
        return ''
    return str(int(string))

for i, row in nc_investor_transactions.iterrows():
    transid = row.transid
    saledate_ordered = row.saledate_ordered
    year = row.year
    month = row.month
    day = row.day
    timestamp = datetime(year, month, day).replace(tzinfo=timezone.utc).timestamp()
    lat = row.lat_rounded
    lng = row.lng_rounded
    address = "{0}, {1}, {2} {3}".format(
        row.propertyfullstreetaddress,
        row.propertycity,
        row.propertystate,
        to_str_int(row.propertyzip)
    )
    investor = row.investor_label_lvl2
    coordinates = [lng, lat]
    
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": coordinates
        },
        "properties": {
            "timestamp": timestamp,
        }
    }
    timeline_geodata['features'].append(feature)
    
timeline_geojson_filepath = os.path.join(DATA_DIRECTORY, 'timeline_geojson.json')

with open(timeline_geojson_filepath, 'w') as f:
    json.dump(timeline_geodata, f)

In [ ]:
timeline_filepath = os.path.join(DATA_DIRECTORY, 'timeline.json')

subprocess.run([
    MAPSHAPER,
    timeline_geojson_filepath,
    "-rename-layers",
    "names=timeline",
    "-o",
    "format=topojson",
    timeline_filepath
], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

### (3) Output cumulative transaction data by day of transaction

Important to get the first and last timestamps for the scrolling map.

In [29]:
import copy

timestamp_year_count = {}
timestamp_year_cumulative = {}
for timestamp in sorted(nc_investor_transactions.timestamp.unique()):
    timestamp_year_count[str(timestamp)] = {}

cumulative_count = {"pre-2010": {"current": 0, "previous": 0, "total": 0}}
per_year_count = {"pre-2010": 0}
for year in sorted(nc_investor_transactions.year.unique()):
    if year < 2010:
        continue
    per_year_count[str(year)] = 0
    cumulative_count[str(year)] = {"current": 0, "previous": 0, "total": 0}

cumulative_total = 0
for i, row in nc_investor_transactions.iterrows():
    year = row.year
    if year < 2010:
        year = "pre-2010"
        
    if cumulative_count[str(year)]["current"] == 0 and year != "pre-2010":
        if year == 2010:
            prev_year = "pre-2010"
        else:
            prev_year = year - 1
        cumulative_count[str(year)]["previous"] = cumulative_total
    
    
    timestamp = row.timestamp
    per_year_count[str(year)] += 1
    cumulative_count[str(year)]["current"] += 1
    cumulative_count[str(year)]["total"] = cumulative_count[str(year)]["current"] + cumulative_count[str(year)]["previous"]
    
    copy_per_year_count = copy.deepcopy(per_year_count)
    copy_cumulative_count = copy.deepcopy(cumulative_count)
    
    timestamp_year_count[str(timestamp)] = copy_per_year_count
    timestamp_year_cumulative[str(timestamp)] = copy_cumulative_count
    cumulative_total += 1


In [30]:
timestamp_keys = sorted(list(timestamp_year_count.keys()))
print("First: {0}, Last: {1}".format(timestamp_keys[0], timestamp_keys[-1]))

First: 1339113600, Last: 1615939200


In [31]:
output_filepath = os.path.join(DATA_DIRECTORY, 'timestamps_cumulative.json')
    
with open(output_filepath, 'w') as f:
    json.dump(timestamp_year_cumulative, f)

In [32]:
output_filepath = os.path.join(DATA_DIRECTORY, 'timestamps_aggregate.json')
    
with open(output_filepath, 'w') as f:
    json.dump(timestamp_year_count, f)